In [22]:
import pandas as pd



def styled_formula_table(df):
    """Return styled display table for formulas"""
    
    mytable = df.style.set_properties(subset=['formula'], **{'width': '200px'}).set_table_styles ([
        {'selector': 'th.col_heading', 'props': 'text-align: left'},
        {'selector': 'td', 'props': 'text-align: left'}])

    return (mytable)
    



if __name__ == '__main__':
    
    # Latex string in csv needs to be enclosed a single $ to enable left align
    df=  pd.read_csv(filepath_or_buffer='formulas.csv')
    display(styled_formula_table(df))
    


,Category,Sub-category,Description,formula,On formula sheet,Comment
0,Indices,Index Rules,Multiplying terms with same base,$a^m \times a^n = a^{m+n}$,False,nan
1,Indices,Index Rules,Dividing terms with same base,$a^m \div a^n = \dfrac{a^m}{a^n} = a^{m-n}$,False,nan
2,Indices,Index Rules,Power of a power,$(a^m)^n = a^{m \times n}$,False,nan
3,Indices,Index Rules,Powers of products,$(ab)^n = a^nb^n$,False,nan
4,Indices,Index Rules,Powers of quotients,$\left(\dfrac{a}{b}\right)^n = \dfrac{a^n}{b^n}$,False,nan
5,Indices,Index Rules,Power of zero,$a^0=1$,False,nan
6,Indices,Index Rules,Negative powers,$a^{-n}=\dfrac{1}{a^n}$,False,nan
7,Indices,Index Rules,Negative powers of quotients,$\left(\dfrac{a}{b}\right)^{-n} = \left(\dfrac{b}{a}\right)^n$,False,nan


In [19]:
from IPython.display import Latex
test = r'$\left(\dfrac{a}{b}\right)^{-n} = \left(\dfrac{b}{a}\right)^n$'
display (Latex(test))

<IPython.core.display.Latex object>